<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #09



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #09

<br>
Facilities <br>
$|I| = 2000$ 
<br>
<br>
Clients <br>
$|J| = 2000$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [ ]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(file_list[i+shift])
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/MT1'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))



start = time.time()


# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()


end = time.time()
print("TIME IS: ",end - start)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4002000 rows, 4002000 columns and 12000000 nonzeros
Model fingerprint: 0x9ddf53af
Variable types: 4000000 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.626577e+08
Presolve removed 0 rows and 0 columns (presolve time = 25s) ...
Presolve removed 0 rows and 0 columns (presolve time = 29s) ...
Presolve removed 0 rows and 0 columns (presolve time = 98s) ...
Presolve removed 0 rows and 0 columns (presolve time = 100s) ...
Presolve removed 0 rows and 0 columns (presolve time = 122s) ...
Presolve removed 0 rows and 0 columns (presolve time = 125s) ...
Presolve removed 0 rows a

In [ ]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_09_Simplex.lp')

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [ ]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random


start = time.time()

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)



end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

In [ ]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)

In [ ]:
len(facs)

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [ ]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [ ]:
#Get the greatest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [ ]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [ ]:
#Get the lowest shipping cost for each Client


start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)